In [3]:
import pandas as pd
import numpy as np

In [71]:
doc_ph = pd.read_csv('data/documenten_ph/DB_V2.txt', encoding='latin_1', sep='\t')

/Users/tomdke/.virtualenvs/py3_data_science/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [140]:
pharma_ref = pd.read_csv('data/documenten_ph/PHARMA_REF.csv', encoding='latin_1', sep=';')

In [141]:
atc = pd.read_csv('data/emd/ATCDPP-r1808.csv', sep=';')

In [142]:
sam = pd.read_csv('data/emd/csv4Emd_Nl_1901A/Sam.csv', sep=';')

In [143]:
stof = pd.read_csv('data/emd/csv4Emd_Nl_1901A/Stof.csv', sep=';')

### Case: what is the correlation between drug use and evidence from clinical trials? How is this evidence correlated with the time it takes for a drug to be brought to the market? Is there a correlation with reimbursements?

In [76]:
## remove unneeded data for memory purposes

doc_ph = doc_ph[
    (doc_ph.Type == "Général") # focus op niet psychiatrische ziekenhuizen
    & (doc_ph.Province == "Brabant Flamand") # arbitrair regio
    & (doc_ph.drug_code != 0) # invalide drug code
#     & (df_raw.hosp_serv_id == 220) # arbitrair 1 departement
    & (doc_ph.realization_date % 10 < 5) # uitzondering niet te herleiden tot 1 van 4 quartalen
    & (doc_ph.realization_date // 10 > 2002)
]

doc_ph.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1723119 entries, 1921729 to 24767314
Data columns (total 11 columns):
account_yy_ss       int64
Province            object
Type                object
hosp_serv_id        object
reimbt_cat_id       int64
drug_code           object
realization_date    int64
quantity            int64
amount_reimb        float64
amount_not_reimb    float64
Year                int64
dtypes: float64(2), int64(5), object(4)
memory usage: 157.8+ MB


#### Goal 1: link drug_code (RIZIV pharma_ref.txt) to molecules (EMD Stof.csv)

In [144]:
## minimize correlation datasets
pharma_ref = pharma_ref[['drug_code', 'code_atc', 'drug_name_aggregated']]
pharma_ref.rename(columns = {'code_atc': 'atc'}, inplace=True)
atc = atc[['mppcv', 'atc', 'atcnm_e']]
sam = sam[['mppcv', 'Stofcv']]
stof = stof[['StofCV', 'nInNm']]
stof.rename(columns = {'StofCV': 'Stofcv'}, inplace = True)

In [145]:
atc['mppcv'] = pd.to_numeric(atc['mppcv'], errors='coerce')
sam_atc = pd.merge(sam, atc, on='mppcv')
sam_stof = pd.merge(sam_atc, stof, on='Stofcv')
sam_stof = sam_stof.drop_duplicates(['atc', 'Stofcv'])
sam_stof = sam_stof.drop(['mppcv'], axis=1)
sam_stof = sam_stof[sam_stof.nInNm != 'geen actieve ingrediënten']

sam_stof.reset_index(drop=True)

,Stofcv,atc,atcnm_e,nInNm
0,1860,Z,atc absent,zolmitriptan
1,1860,N02CC03,zolmitriptan,zolmitriptan
2,1205,R03DC03,montelukast,montelukast natrium
3,1205,Z,atc absent,montelukast natrium
4,634,N06AB10,escitalopram,escitalopram oxalaat
5,1613,A05BA03,silymarin,Silybum marianum (silymarine)
6,1418,M01AC01,piroxicam,piroxicam
7,1418,M02AA07,piroxicam,piroxicam
8,1037,Z,atc absent,levodopa
9,1037,N04BA03,"levodopa, decarboxylase inhibitor and comt inh...",levodopa


In [148]:
pharma_stof = pd.merge(pharma_ref, sam_stof, on='atc')

pharma_stof.to_csv('data/merged/drug_code__stof.csv')